In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import re
import regex

In [ ]:
!pip install regex

In [ ]:
headers= {'Accept-Encoding':'gzip, deflate',
          'Accept-Language':'en-US,en;q=0.9',
          'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'}     

url = 'https://www.website.com/startups'     

In [ ]:
response = requests.get(url,headers = headers)

In [ ]:
response.status_code

In [ ]:
soup = BeautifulSoup(response.content)

## 1. Get Industry list and urls

We get the full list of every field segment for the start ups on the web we want to scrappe.

In [ ]:
pd.set_option('display.max_rows',20000)

In [ ]:
box = soup.find_all('div',attrs={'class':'card-title-programmatic-seo'})

In [ ]:
box[390].get_text()

In [ ]:
len(box)

In [ ]:
industry_list=[]

for i in range(390,614):
  
  try:

    name=box[i].get_text()
    print(name)

    data = {'Field':name}
    industry_list.append(data)
  
  except:
    pass

In [ ]:
industry_n=pd.DataFrame(industry_list)

In [ ]:
industry_n

In [ ]:
urls = soup.find_all('a',attrs={'class':'card-programmatic-seo w-inline-block'})

In [ ]:
len(urls)

In [ ]:
urls[0].get('href')

In [ ]:
url_industry_list=[]

for i in range(390,614):
  
  try:

    link=('https://www.website.com'+urls[i].get('href'))

    print(link)

    url_industry_list.append(link)
  
  except:
    pass

## 2. Get General info from every Marketing Start up

In [ ]:
link = 'https://www.website.com/startups/drones'

In [ ]:
response = requests.get(link, headers=headers)
soup = BeautifulSoup(response.content)

### Get company name

In [ ]:
box = soup.find_all('div',attrs={'class':'_0-padding-mobile w-col w-col-8 w-col-stack'})

In [ ]:
box[0]

In [ ]:
titles = box[0].find_all('h3')

In [ ]:
titles

In [ ]:
company_n_list = []

for title in titles:
  company_n = title.get_text().split(" ", 1)[1]
  data = {'Company Name':company_n}
  company_n_list.append(data)
  print(company_n)

In [ ]:
company_n=pd.DataFrame(company_n_list)

In [ ]:
company_n['Field']='Marketing'

In [ ]:
company_n

### Get company Country, City, Started, Nº of employees, Funding amount, Number of funding rounds, Number of investors

In [ ]:
boxes = soup.find_all('ul',attrs={'role':'list'})

In [ ]:
boxes[0]

In [ ]:
def convert_to_dollars(funding_string):
    funding_string = str(funding_string)
    currency_regex = r"([A-Z]{0,2}\p{Sc})"
    matches = regex.findall(currency_regex, funding_string)
    currency_symbol = matches[0] if matches else ""
    if "SGD" in funding_string:
        return int(regex.sub(r'[^\d]', '', funding_string))/1.35  # assuming 1 USD = 1.35 SGD
    elif "SEK" in funding_string:
        return int(regex.sub(r'[^\d]', '', funding_string))/10.51  # assuming 1 USD = 10.51 SEK
    elif "DKK" in funding_string:
        return int(regex.sub(r'[^\d]', '', funding_string))/7.06
    elif "PLN" in funding_string:
        return int(regex.sub(r'[^\d]', '', funding_string))/4.48
    elif "PKR" in funding_string:
        return int(regex.sub(r'[^\d]', '', funding_string))/257.79
    elif "RON" in funding_string:
        return int(regex.sub(r'[^\d]', '', funding_string))/4.66
    elif "NOK" in funding_string:
        return int(regex.sub(r'[^\d]', '', funding_string))/10.39
    elif "MIR" in funding_string:
        return int(regex.sub(r'[^\d]', '', funding_string))/4.44
    elif "THB" in funding_string:
        return int(regex.sub(r'[^\d]', '', funding_string))/34.92
    elif "RUB" in funding_string:
        return int(regex.sub(r'[^\d]', '', funding_string))/76.02
    elif "AED" in funding_string:
        return int(regex.sub(r'[^\d]', '', funding_string))/3.67
    else:
      if "$" in currency_symbol:
          return int(regex.sub(r'[^\d]', '', funding_string))
      elif "€" in currency_symbol:
          return int(regex.sub(r'[^\d]', '', funding_string))
      elif "₹" in currency_symbol: 
          return int(regex.sub(r'[^\d]', '', funding_string))/82.93  # assuming 1 USD = 82.93 INR
      elif "₩" in currency_symbol: 
          return int(regex.sub(r'[^\d]', '', funding_string))/1314.80  # assuming 1 USD = 1314.80 ₩
      elif "£" in currency_symbol: 
          return int(regex.sub(r'[^\d]', '', funding_string))/0.84
      elif "¥" in currency_symbol:
          if "CN¥" in funding_string:
              return int(regex.sub(r'[^\d]', '', funding_string))/6.96  # assuming 1 USD = 6.96 CNY
          else:
              return int(regex.sub(r'[^\d]', '', funding_string))/136.47  # assuming 1 USD = 136.47 JPY
      else:
          return None  # invalid currency symbol

In [ ]:
funding_string = "Funding amount: SGD863,000,000"

print(int(regex.sub(r'[^\d]', '', funding_string))/1.35)

In [ ]:
# Example usage:
funding_string1 = "Funding amount: ¥53,500,002,780"
dollars1 = convert_to_dollars(funding_string1)
print(dollars1)  

funding_string2 = "Funding amount: CN¥500,000"
dollars2 = convert_to_dollars(funding_string2)
print(dollars2)  

# Example usage:
funding_string = "Funding amount: $863,000,000"
dollars = convert_to_dollars(funding_string)
print(dollars)  

# Example usage:
funding_string = "Funding amount: €863,000,000"
dollars = convert_to_dollars(funding_string)
print(dollars)  

In [ ]:
# We create empty lists for filling once we scrappe the web
company_c_list = []
company_city_list = []
company_employee_list_min = []
company_employee_list_max = []
company_fund_a_list = []
company_fund_r_list = []
company_year_list = []
company_inv_list = []

# Boxes is the full list of company for the specific URL. So we create a loop to check every company
for box in boxes:

  # We fill every future variable with an 'empty' value, so if it does not change later on, means that the company does not have the specific value
  company_c = ' '
  company_city = ' '
  year = ' '
  employees = ' '
  funding_amount = ' '
  company_fund_rounds = ' '
  investors = ' '

  try:
    print('New company')
    
    company_data = box.find_all('li')

    # On every company we try to find every field (Country, City, etc.)
    for data in company_data:
      word = data.get_text()
      if 'Country: ' in word: 
        
        company_c = word.replace('Country: ','')
        print(company_c)

        data = {'Country':company_c}
        company_c_list.append(data)

      elif 'City: ' in word: 
        
        company_city = word.replace('City: ','')
        print(company_city)
        data = {'City':company_city}
        company_city_list.append(data)

      elif 'Started in: ' in word: 
        
        print(word)
        year = word.replace('Started in: ','')
        year = int(year)
        print(year)
        data = {'Started in':year}
        company_year_list.append(data)

      elif 'Number of employees: ' in word: 
        
        if '-' in word:

          employees = word.replace('Number of employees: ','')
          print(employees)
          num1, num2 = [int(num) for num in employees.split('-')]

          print(num1)
          print(num2)

          data1 = {'Employee Min':num1}
          data2 = {'Employee Max':num2}
          
          company_employee_list_min.append(data1)
          company_employee_list_max.append(data2)
        
        elif '+' in word:

          employees = word.replace('Number of employees: ','')
          employees = employees.replace('+','')
          print(employees)
          num1 = int(employees)
          data1 = {'Employee Min':num1}
          data2 = {'Employee Max':num1}
                  
          company_employee_list_min.append(data1)
          company_employee_list_max.append(data2)

      elif 'Funding amount: ' in word: 
        
        print(word)
        funding_amount = re.findall("\d+", word)
        funding_amount = "".join(map(str, funding_amount))
        funding_amount = int(funding_amount)
        
        print(funding_amount)

        data = {'Funding Amount':word}
        company_fund_a_list.append(data)

      elif 'Number of funding rounds: ' in word: 
        
        company_fund_rounds = word.replace('Number of funding rounds: ','')
        company_fund_rounds = int(company_fund_rounds)
        print(company_fund_rounds)
        data = {'Nº of Funding Rounds':company_fund_rounds}
        company_fund_r_list.append(data)


      elif 'Number of investors: ' in word: 
        
        investors = word.replace('Number of investors: ','')
        investors = int(investors)
        print(investors)
        data = {'Nº of investors':investors}
        company_inv_list.append(data)
                                
    # If we do not find info from a field, we give it an 'empty' value
    if company_c == ' ':

      data = {'Country':'Empty'}
      company_c_list.append(data)

    if company_city == ' ':

      data = {'Country':'Empty'}
      company_c_list.append(data)

    if year == ' ':

      data = {'Started in':np.nan}
      company_year_list.append(data)

    if employees == ' ':
      
      data1 = {'Employee Min':np.nan}
      data2 = {'Employee Max':np.nan}

      company_employee_list_min.append(data1)
      company_employee_list_max.append(data2)

    if funding_amount == ' ':

      data = {'Funding Amount':np.nan}
      company_fund_a_list.append(data)

    if company_fund_rounds == ' ':

      data = {'Nº of Funding Rounds':np.nan}
      company_fund_r_list.append(data)

    if investors == ' ':
      
      data = {'Nº of investors':np.nan}
      company_inv_list.append(data)


  except:
    pass


In [ ]:
print(len(company_c_list))
print(len(company_city_list))
print(len(company_year_list))
print(len(company_employee_list_max))
print(len(company_fund_a_list))
print(len(company_fund_r_list))
print(len(company_year_list))
print(len(company_inv_list))

In [ ]:
company_n['Country']=pd.DataFrame(company_c_list)
company_n['City']=pd.DataFrame(company_city_list)
company_n['Started in']=pd.DataFrame(company_year_list)
company_n['Employees Min']=pd.DataFrame(company_employee_list_min)
company_n['Employees Max']=pd.DataFrame(company_employee_list_max)
company_n['Funding Amount ($)']=pd.DataFrame(company_fund_a_list)
company_n['Nº Funding Rounds']=pd.DataFrame(company_fund_r_list)
company_n['Nº Investors']=pd.DataFrame(company_inv_list)

In [ ]:
company_n

In [ ]:
company_n.dtypes

In [ ]:
company_n['Nº Investors'].isna().sum()

##A loop for every field

In [ ]:
url_industry_list

In [ ]:
# We create empty lists for filling once we scrappe the web
company_n_list = []
company_f_list = []
company_c_list = []
company_city_list = []
company_employee_list_min = []
company_employee_list_max = []
company_fund_a_list = []
company_fund_r_list = []
company_year_list = []
company_inv_list = []

for link in url_industry_list:
  time.sleep(2)

  print(link)

  time.sleep(5)  
  try:
    
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.content)
    response.status_code 
    
    box = soup.find_all('div',attrs={'class':'_0-padding-mobile w-col w-col-8 w-col-stack'})

    titles = box[0].find_all('h3')

    for title in titles:
      company_n = title.get_text().split(" ", 1)[1]
      data = {'Company Name':company_n}
      company_n_list.append(data)
      print(company_n)

      field = link.split('/')[-1]
      print(field)
      data = {'Field':field}
      company_f_list.append(data)     

    boxes = soup.find_all('ul',attrs={'role':'list'})

    # Boxes is the full list of company for the specific URL. So we create a loop to check every company
    for box in boxes:

      # We fill every future variable with an 'empty' value, so if it does not change later on, means that the company does not have the specific value
      company_c = ' '
      company_city = ' '
      year = ' '
      employees = ' '
      funding_amount = ' '
      company_fund_rounds = ' '
      investors = ' '

      try:
        print('New company')
        
        company_data = box.find_all('li')

        # On every company we try to find every field (Country, City, etc.)
        for data in company_data:
          word = data.get_text()
          if 'Country: ' in word: 
            
            company_c = word.replace('Country: ','')
            print(company_c)

            data = {'Country':company_c}
            company_c_list.append(data)

          elif 'City: ' in word: 
            
            company_city = word.replace('City: ','')
            print(company_city)
            data = {'City':company_city}
            company_city_list.append(data)

          elif 'Started in: ' in word: 
            
              if '#N/A' not in word:
                print(word)
                year = word.replace('Started in: ','')
                year = int(year)
                print(year)
                data = {'Started in':year}
                company_year_list.append(data)

          elif 'Number of employees: ' in word: 
            
            if '-' in word:

              employees = word.replace('Number of employees: ','')
              print(employees)
              num1, num2 = [int(num) for num in employees.split('-')]

              print(num1)
              print(num2)

              data1 = {'Employee Min':num1}
              data2 = {'Employee Max':num2}
              
              company_employee_list_min.append(data1)
              company_employee_list_max.append(data2)
            
            elif '+' in word:

              employees = word.replace('Number of employees: ','')
              employees = employees.replace('+','')
              print(employees)
              num1 = int(employees)
              data1 = {'Employee Min':num1}
              data2 = {'Employee Max':num1}
                      
              company_employee_list_min.append(data1)
              company_employee_list_max.append(data2)

          elif 'Funding amount: ' in word: 
            
            print(word)

            funding_amount = re.findall("\d+", word)
            funding_amount = "".join(map(str, funding_amount))
            funding_amount = int(funding_amount)
            
            print(funding_amount)

            data = {'Funding Amount':word}
            company_fund_a_list.append(data)

          elif 'Number of funding rounds: ' in word: 
            
            company_fund_rounds = word.replace('Number of funding rounds: ','')
            company_fund_rounds = int(company_fund_rounds)
            print(company_fund_rounds)
            data = {'Nº of Funding Rounds':company_fund_rounds}
            company_fund_r_list.append(data)


          elif 'Number of investors: ' in word: 
            
            investors = word.replace('Number of investors: ','')
            investors = int(investors)
            print(investors)
            data = {'Nº of investors':investors}
            company_inv_list.append(data)
                                    
        # If we do not find info from a field, we give it an 'empty' value
        if company_c == ' ':

          data = {'Country':'Empty'}
          company_c_list.append(data)

        if company_city == ' ':

          data = {'Country':'Empty'}
          company_c_list.append(data)

        if year == ' ':

          data = {'Started in':np.nan}
          company_year_list.append(data)

        if employees == ' ':
          
          data1 = {'Employee Min':np.nan}
          data2 = {'Employee Max':np.nan}

          company_employee_list_min.append(data1)
          company_employee_list_max.append(data2)

        if funding_amount == ' ':

          data = {'Funding Amount':np.nan}
          company_fund_a_list.append(data)

        if company_fund_rounds == ' ':

          data = {'Nº of Funding Rounds':np.nan}
          company_fund_r_list.append(data)

        if investors == ' ':
          
          data = {'Nº of investors':np.nan}
          company_inv_list.append(data)


      except:
        pass


  except:
    pass

In [ ]:
print(len(company_n_list))
print(len(company_f_list))
print(len(company_c_list))
print(len(company_city_list))
print(len(company_year_list))
print(len(company_employee_list_max))
print(len(company_fund_a_list))
print(len(company_fund_r_list))
print(len(company_year_list))
print(len(company_inv_list))

In [ ]:
company_table = pd.DataFrame()

company_table['Company Name']=pd.DataFrame(company_n_list)
company_table['Field']=pd.DataFrame(company_f_list)
company_table['Country']=pd.DataFrame(company_c_list)
company_table['City']=pd.DataFrame(company_city_list)
company_table['Started in']=pd.DataFrame(company_year_list)
company_table['Employees Min']=pd.DataFrame(company_employee_list_min)
company_table['Employees Max']=pd.DataFrame(company_employee_list_max)
company_table['Funding Amount ($)']=pd.DataFrame(company_fund_a_list)
company_table['Nº Funding Rounds']=pd.DataFrame(company_fund_r_list)
company_table['Nº Investors']=pd.DataFrame(company_inv_list)

In [ ]:
company_table

In [ ]:
company_table["Funding Amount Number ($)"] = company_table["Funding Amount ($)"].apply(convert_to_dollars)

In [ ]:
company_table.to_excel('start_up_overallv1.xlsx',index=False)

In [ ]:
company_table.to_csv('output.csv',index=False)

In [ ]:
company_table.dtypes